In [3]:
%config IPCompleter.greedy=True

import pandas as pd

In [4]:
YEARS = range(2011,2022)

In [6]:
data = pd.DataFrame()

for i in YEARS:
    print(i)
    i_data = pd.read_csv('https://github.com/guga31bb/nflfastR-data/blob/master/data/' \
                         'play_by_play_' + str(i) + '.csv.gz?raw=True',
                         compression='gzip', low_memory=False)
    
    data = data.append(i_data, sort=True)
    
    data.columns = data.columns.str.replace("roof_type", "roof")

data.loc[(data.passer == 'Aa.Rodgers'), 'passer'] = 'A.Rodgers'
    
data = data.loc[(data.play_type.isin(['no_play','pass','run'])) & (data.epa.isna()==False)]

post_data = data.loc[data.season_type=='POST']

reg_data = data.loc[data.season_type=='REG']

data = data.loc[data.season_type=='REG']

2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021


In [7]:
data.columns = data.columns.str.replace("Roof", "roof")

indoor_data = data.loc[data['roof'] != 'outdoors']

outdoor_data = data.loc[data['roof'] == 'outdoors']

In [9]:
qb_indoor_data = indoor_data.groupby(['passer'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                      'play_id':'count'})
qb_indoor_data = qb_indoor_data.loc[qb_indoor_data.play_id>200]

qb_indoor_data = qb_indoor_data.round(2)

qb_indoor_data.columns = ['Player', 'EPA per Dropback','CPOE','Dropbacks']

qb_indoor_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_indoor_data

,Player,EPA per Dropback,CPOE,Dropbacks
109,J.Garoppolo,0.36,2.17,226
93,J.Allen,0.30,6.95,266
175,P.Mahomes,0.29,-0.64,324
208,T.Brady,0.27,3.34,720
13,A.Rodgers,0.26,3.14,1316
...,...,...,...,...
20,B.Gabbert,-0.13,-6.82,436
227,T.Savage,-0.13,-4.03,229
128,J.Skelton,-0.14,-5.82,360
126,J.Rosen,-0.31,-10.30,365


In [10]:
qb_outdoor_data = outdoor_data.groupby(['passer'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                        'play_id':'count'})
qb_outdoor_data = qb_outdoor_data.loc[qb_outdoor_data.play_id>100]

qb_outdoor_data = qb_outdoor_data.round(2)

qb_outdoor_data.columns = ['Player', 'EPA per Dropback','CPOE','Dropbacks']

qb_outdoor_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_outdoor_data

,Player,EPA per Dropback,CPOE,Dropbacks
264,P.Mahomes,0.30,3.61,2361
265,P.Manning,0.25,4.76,2038
195,K.Murray,0.25,4.49,671
12,A.Rodgers,0.24,3.63,5263
305,T.Brady,0.22,1.21,6592
...,...,...,...,...
165,J.Rosen,-0.31,-9.47,262
269,P.Walker,-0.36,-10.13,107
52,C.Hanie,-0.43,-7.21,134
274,R.Finley,-0.44,-13.22,153


In [11]:
qb_weather_split_data = qb_indoor_data.merge(qb_outdoor_data, on='Player')

qb_weather_split_data['IN vs OUT'] = qb_weather_split_data['EPA per Dropback_x'] - qb_weather_split_data['EPA per Dropback_y']

qb_weather_split_data.columns = ['Player', 'IN EPA/dropback','IN CPOE','IN Dropbacks', 'OUT EPA/dropback','OUT CPOE','OUT Dropbacks', 'IN EPA/OUT EPA Diff']

qb_weather_split_data.sort_values('IN EPA/OUT EPA Diff', ascending=False, inplace=True)

pd.set_option('display.max_rows', 100)

In [12]:
qb_weather_split_data.head(25)

,Player,IN EPA/dropback,IN CPOE,IN Dropbacks,OUT EPA/dropback,OUT CPOE,OUT Dropbacks,IN EPA/OUT EPA Diff
39,K.Clemens,0.05,-5.46,282,-0.30,-3.60,140,0.35
7,K.Allen,0.22,2.74,213,-0.04,-0.95,534,0.26
25,B.Weeden,0.12,-0.80,262,-0.07,-3.90,849,0.19
9,R.Griffin III,0.21,3.95,234,0.02,0.14,1338,0.19
0,J.Garoppolo,0.36,2.17,226,0.19,1.86,1384,0.17
1,J.Allen,0.30,6.95,266,0.13,0.89,2081,0.17
41,D.Stanton,0.05,-5.57,328,-0.11,-10.55,227,0.16
67,J.Skelton,-0.14,-5.82,360,-0.30,-10.34,188,0.16
33,S.Darnold,0.07,0.06,216,-0.07,-3.01,1622,0.14
11,M.Mariota,0.18,2.73,399,0.06,0.60,1794,0.12


In [13]:
qb_weather_split_data.tail(25)

,Player,IN EPA/dropback,IN CPOE,IN Dropbacks,OUT EPA/dropback,OUT CPOE,OUT Dropbacks,IN EPA/OUT EPA Diff
12,J.Herbert,0.17,-1.29,871,0.19,3.16,498,-0.02
50,N.Foles,0.00,-0.37,624,0.02,0.55,1716,-0.02
58,K.Kolb,-0.04,-4.50,395,-0.02,-5.70,144,-0.02
56,B.Hoyer,-0.02,-2.50,612,0.00,-3.95,1033,-0.02
54,S.Bradford,-0.01,-1.64,1416,0.01,-0.61,1309,-0.02
47,M.Hasselbeck,0.02,-1.35,427,0.04,-0.75,752,-0.02
35,A.Smith,0.07,-0.26,645,0.10,0.54,3809,-0.03
15,M.Ryan,0.15,2.31,4758,0.18,3.69,2694,-0.03
59,J.Freeman,-0.05,-4.34,327,-0.02,-3.97,1148,-0.03
38,J.Goff,0.05,0.20,1085,0.09,-1.39,2209,-0.04


In [14]:
qb_reg_data = reg_data.groupby(['passer'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                      'play_id':'count'})
qb_reg_data = qb_reg_data.loc[qb_reg_data.play_id>200]

qb_reg_data = qb_reg_data.round(2)

qb_reg_data.columns = ['Player', 'EPA per Dropback','CPOE','Dropbacks']

qb_reg_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_reg_data

,Player,EPA per Dropback,CPOE,Dropbacks
304,P.Mahomes,0.30,3.10,2685
305,P.Manning,0.24,4.47,2383
17,A.Rodgers,0.24,3.53,6579
353,T.Brady,0.22,1.42,7312
176,J.Garoppolo,0.21,1.90,1610
...,...,...,...,...
42,B.Petty,-0.22,-10.09,289
391,Z.Mettenberger,-0.24,-2.34,396
44,B.Quinn,-0.29,-4.21,239
203,J.Rosen,-0.31,-9.95,627


In [15]:
qb_post_data = post_data.groupby(['passer'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                      'play_id':'count'})
qb_post_data = qb_post_data.loc[qb_post_data.play_id>50]

qb_post_data = qb_post_data.round(2)

qb_post_data.columns = ['Player', 'EPA per Dropback','CPOE','Dropbacks']

qb_post_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_post_data

,Player,EPA per Dropback,CPOE,Dropbacks
52,N.Foles,0.32,5.05,224
31,J.Flacco,0.31,2.03,303
54,P.Mahomes,0.28,-0.30,364
73,T.Tebow,0.28,-12.53,59
71,T.Romo,0.27,2.57,67
16,C.Kaepernick,0.26,0.46,196
48,M.Ryan,0.26,6.24,314
5,A.Rodgers,0.23,3.14,663
63,T.Brady,0.23,0.03,1174
56,P.Rivers,0.21,0.63,196


In [16]:
qb_playoff_split_data = qb_reg_data.merge(qb_post_data, on='Player')

qb_playoff_split_data['REG vs POST'] = qb_playoff_split_data['EPA per Dropback_x'] - qb_playoff_split_data['EPA per Dropback_y']

qb_playoff_split_data.columns = ['Player', 'REG EPA/dropback','REG CPOE','REG Dropbacks', 'POST EPA/dropback','POST CPOE','POST Dropbacks', 'REG EPA/POST EPA Diff']

qb_playoff_split_data.sort_values('REG EPA/POST EPA Diff', ascending=False, inplace=True)

pd.set_option('display.max_rows', 100)

In [17]:
qb_playoff_split_data.head(25)

,Player,REG EPA/dropback,REG CPOE,REG Dropbacks,POST EPA/dropback,POST CPOE,POST Dropbacks,REG EPA/POST EPA Diff
26,A.Dalton,0.07,-0.05,5729,-0.23,-5.43,184,0.30
8,L.Jackson,0.18,0.86,1664,-0.08,-3.16,172,0.26
15,K.Cousins,0.13,2.63,4786,-0.13,-3.37,133,0.26
1,P.Manning,0.24,4.47,2383,-0.01,0.62,345,0.25
17,C.Palmer,0.11,1.79,3513,-0.12,-1.57,89,0.23
6,D.Watson,0.20,4.19,2230,0.00,-2.11,159,0.20
4,J.Garoppolo,0.21,1.90,1610,0.03,-2.94,68,0.18
35,B.Osweiler,-0.02,-2.97,1351,-0.19,-7.39,79,0.17
13,J.Allen,0.15,1.56,2347,0.02,-0.33,193,0.13
25,J.Goff,0.07,-0.86,3294,-0.03,-6.36,221,0.10


In [18]:
qb_playoff_split_data.tail(10)

,Player,REG EPA/dropback,REG CPOE,REG Dropbacks,POST EPA/dropback,POST CPOE,POST Dropbacks,REG EPA/POST EPA Diff
36,T.Yates,-0.09,-5.89,393,-0.03,-8.59,57,-0.06
20,B.Mayfield,0.10,0.03,2238,0.18,-3.35,80,-0.08
7,T.Romo,0.18,3.35,2557,0.27,2.57,67,-0.09
11,M.Ryan,0.16,2.82,7452,0.26,6.24,314,-0.10
28,E.Manning,0.06,-0.78,5377,0.20,1.64,225,-0.14
33,B.Bortles,-0.00,-4.62,3142,0.16,-4.80,97,-0.16
23,C.Kaepernick,0.09,-2.07,2153,0.26,0.46,196,-0.17
30,J.Flacco,0.04,-0.88,5311,0.31,2.03,303,-0.27
32,N.Foles,0.01,0.30,2340,0.32,5.05,224,-0.31
37,T.Tebow,-0.12,-7.53,362,0.28,-12.53,59,-0.40


In [19]:
qb_number_data = data.loc[~data['passer_player_name'].isin(['T.Brady', 'A.Rodgers', 'P.Mahomes', 'P.Manning', 'D.Brees'])]

qb_number_data = qb_number_data.groupby(['passer_jersey_number'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                             'play_id':'count'})
qb_number_data = qb_number_data.loc[qb_number_data.play_id>200]

qb_number_data = qb_number_data.round(2)

qb_number_data.columns = ['Jersey Number', 'EPA per Dropback','CPOE','Dropbacks']

qb_number_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_number_data

,Jersey Number,EPA per Dropback,CPOE,Dropbacks
11,12.0,0.14,-1.30,8331
17,18.0,0.11,-0.30,739
16,17.0,0.11,1.41,15908
8,9.0,0.10,0.09,14667
3,4.0,0.09,0.76,14284
0,1.0,0.07,-0.69,8909
1,2.0,0.07,0.07,12590
9,10.0,0.07,-0.49,14712
2,3.0,0.07,1.28,17827
7,8.0,0.06,0.49,18777


In [20]:
qb_number_e_data = data.groupby(['passer_jersey_number'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                             'play_id':'count'})
qb_number_e_data = qb_number_e_data.loc[qb_number_e_data.play_id>200]

qb_number_e_data = qb_number_e_data.round(2)

qb_number_e_data.columns = ['Jersey Number', 'EPA per Dropback','CPOE','Dropbacks']

qb_number_e_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_number_e_data

,Jersey Number,EPA per Dropback,CPOE,Dropbacks
17,18.0,0.20,3.54,2978
11,12.0,0.17,1.08,20696
14,15.0,0.14,0.60,5358
8,9.0,0.13,1.38,20609
16,17.0,0.11,1.41,15908
3,4.0,0.09,0.76,14284
0,1.0,0.07,-0.69,8909
1,2.0,0.07,0.07,12590
9,10.0,0.07,-0.49,14712
2,3.0,0.07,1.28,17827


In [21]:
qb_num_diff_data = qb_number_data.merge(qb_number_e_data, on='Jersey Number')

qb_num_diff_data['Legacy vs Bums'] = qb_num_diff_data['EPA per Dropback_y'] - qb_num_diff_data['EPA per Dropback_x']

qb_num_diff_data.columns = ['Jersey Number', 'Bum EPA/dropback','Bum CPOE','Bum Dropbacks', 'Legacy EPA/dropback','Legacy CPOE','Legacy Dropbacks', 'Bum EPA/Legacy EPA Diff']

qb_num_diff_data.sort_values('Bum EPA/Legacy EPA Diff', ascending=False, inplace=True)

pd.set_option('display.max_rows', 100)

qb_num_diff_data

,Jersey Number,Bum EPA/dropback,Bum CPOE,Bum Dropbacks,Legacy EPA/dropback,Legacy CPOE,Legacy Dropbacks,Bum EPA/Legacy EPA Diff
12,15.0,0.04,-1.95,2993,0.14,0.60,5358,0.10
1,18.0,0.11,-0.30,739,0.20,3.54,2978,0.09
0,12.0,0.14,-1.30,8331,0.17,1.08,20696,0.03
3,9.0,0.10,0.09,14667,0.13,1.38,20609,0.03
11,14.0,0.04,-1.03,13084,0.04,-1.03,13084,0.00
17,6.0,0.00,-1.49,9086,0.00,-1.49,9086,0.00
16,16.0,0.01,-1.67,5167,0.01,-1.67,5167,0.00
15,13.0,0.01,-1.55,1821,0.01,-1.55,1821,0.00
14,5.0,0.02,-1.86,16216,0.02,-1.86,16216,0.00
13,7.0,0.03,-1.01,21798,0.03,-1.01,21798,0.00


In [22]:
qb_rec_connections = data.groupby(['season', 'posteam', 'receiver_player_name', 'passer_player_name'], as_index=False).agg({'qb_epa':'mean',                                                                      'cpoe':'mean',
                                                                                                                        'play_id':'count'})
qb_rec_connections = qb_rec_connections.round(2)

#qb_rec_connections = qb_rec_connections.loc[qb_rec_connections.play_id > 40]

qb_rec_connections.columns = ['Season', 'Team', 'Receiver', 'Passer', 'EPA per Dropback','CPOE','Dropbacks']

qb_rec_connections.sort_values('EPA per Dropback', ascending=False, inplace=True)

qb_rec_connections

,Season,Team,Receiver,Passer,EPA per Dropback,CPOE,Dropbacks
6516,2018,TEN,D.Cruikshank,K.Byard,7.65,51.26,1
1101,2012,DET,T.Young,Sh.Hill,6.60,72.66,1
5871,2018,CAR,C.Manhertz,C.McCaffrey,6.55,41.28,1
991,2012,CIN,A.Green,M.Sanu,6.47,66.58,1
8205,2020,PHI,D.Jackson,J.Hurts,6.44,65.71,1
...,...,...,...,...,...,...,...
2657,2014,DET,J.Thompson,M.Stafford,-7.49,-54.47,1
4536,2016,LA,T.Gurley,P.Cooper,-7.61,-53.15,1
8460,2021,ATL,R.Gage,J.Rosen,-7.95,-73.87,1
7157,2019,NE,B.Bolden,J.Stidham,-7.98,-62.35,1


In [23]:
qb_other_connections = data.groupby(['season', 'posteam', 'passer_player_name', 'receiver_player_name'], as_index=False).agg({'qb_epa':'mean', 'cpoe':'mean','play_id':'count'})
qb_other_connections = qb_other_connections.round(2)

#qb_other_connections = qb_other_connections.loc[qb_other_connections.play_id > 5]

qb_other_connections.columns = ['Season', 'Team', 'Passer', 'Receiver', 'EPA per Dropback','CPOE','Dropbacks']

qb_other_connections.sort_values('EPA per Dropback', ascending=False, inplace=True)


In [24]:
# MODERN NFL DIFFERENTIALS

rec_vs_others = qb_rec_connections.merge(qb_other_connections, on='Season')

rec_vs_others = rec_vs_others.loc[rec_vs_others['Season'] > 2015]

rec_vs_others = rec_vs_others.loc[rec_vs_others['Passer_x'] == rec_vs_others['Passer_y']]

rec_vs_others = rec_vs_others.loc[rec_vs_others['Receiver_x'] != rec_vs_others['Receiver_y']]

rec_vs_others = rec_vs_others.loc[rec_vs_others['Dropbacks_x'] > 40]
#rec_vs_others = rec_vs_others.loc[rec_vs_others['Dropbacks_y'] > 5]

rec_vs_others['Other Total EPA'] = rec_vs_others['Dropbacks_y'] * rec_vs_others['EPA per Dropback_y']

rec_vs_others = rec_vs_others.groupby(['Season', 'Team_x', 'Receiver_x', 'Passer_y', 'Dropbacks_x', ], as_index=False).agg({'EPA per Dropback_x':'mean', 'Other Total EPA':'sum','Dropbacks_y': 'sum'}) 

rec_vs_others['Other EPA per Dropback'] = rec_vs_others['Other Total EPA']/rec_vs_others['Dropbacks_y']

rec_vs_others['Receiver EPA/Other Receiver EPA Differential'] = rec_vs_others['EPA per Dropback_x'] - rec_vs_others['Other EPA per Dropback']

rec_vs_others = rec_vs_others.round(2)

rec_vs_others.columns = ['Season', 'Team', 'Receiver', 'QB', 'Dropbacks', 'Receiver/QB EPA', 'Rest of Team/QB Total EPA', 'Rest of Team Dropbacks', 'Rest of Team/QB EPA per dropback', 'Differential']

rec_vs_others = rec_vs_others[['Season', 'Team', 'Receiver', 'QB', 'Receiver/QB EPA', 'Dropbacks', 'Rest of Team/QB EPA per dropback', 'Rest of Team Dropbacks', 'Differential']]
rec_vs_others.sort_values('Differential', ascending=False, inplace=True)

rec_vs_others.head(20)

,Season,Team,Receiver,QB,Receiver/QB EPA,Dropbacks,Rest of Team/QB EPA per dropback,Rest of Team Dropbacks,Differential
555,2019,WAS,T.McLaurin,C.Keenum,0.88,42,-0.02,200,0.90
72,2016,LA,K.Britt,C.Keenum,0.73,64,-0.16,255,0.89
406,2018,SEA,T.Lockett,R.Wilson,1.05,71,0.29,330,0.76
456,2019,DEN,C.Sutton,J.Flacco,0.72,59,-0.00,192,0.72
564,2020,ATL,J.Jones,M.Ryan,0.83,67,0.20,539,0.63
535,2019,PIT,D.Johnson,M.Rudolph,0.53,48,-0.07,227,0.60
549,2019,TB,C.Godwin,J.Winston,0.72,121,0.12,486,0.60
70,2016,KC,T.Kelce,A.Smith,0.66,106,0.06,376,0.60
191,2017,GB,D.Adams,B.Hundley,0.39,71,-0.18,242,0.57
479,2019,JAX,D.Chark,G.Minshew,0.66,42,0.09,146,0.57


In [25]:
rec_vs_others.tail(20)

,Season,Team,Receiver,QB,Receiver/QB EPA,Dropbacks,Rest of Team/QB EPA per dropback,Rest of Team Dropbacks,Differential
166,2017,CHI,T.Cohen,M.Trubisky,-0.33,42,0.10,285,-0.43
570,2020,BUF,D.Singletary,J.Allen,0.01,50,0.45,499,-0.44
604,2020,GB,A.Jones,A.Rodgers,0.09,63,0.53,438,-0.44
6,2016,ATL,D.Freeman,M.Ryan,0.06,66,0.50,465,-0.44
794,2021,PHI,J.Reagor,J.Hurts,-0.18,52,0.26,335,-0.44
703,2021,ATL,M.Davis,M.Ryan,-0.22,53,0.23,440,-0.45
584,2020,CIN,A.Green,J.Burrow,-0.09,75,0.38,317,-0.47
541,2019,SEA,J.Hollister,R.Wilson,-0.06,59,0.42,436,-0.48
550,2019,TB,D.Ogunbowale,J.Winston,-0.22,45,0.28,562,-0.50
784,2021,NE,J.Smith,M.Jones,-0.25,44,0.26,409,-0.51


In [27]:
# EAGLES DATA ONLY 

rec_vs_others = qb_rec_connections.merge(qb_other_connections, on='Season')

rec_vs_others = rec_vs_others.loc[rec_vs_others['Season'] > 2015]
rec_vs_others = rec_vs_others.loc[rec_vs_others['Team_x'] == 'PHI']

rec_vs_others = rec_vs_others.loc[rec_vs_others['Passer_x'] == rec_vs_others['Passer_y']]

rec_vs_others = rec_vs_others.loc[rec_vs_others['Receiver_x'] != rec_vs_others['Receiver_y']]

rec_vs_others = rec_vs_others.loc[rec_vs_others['Dropbacks_x'] > 30]
#rec_vs_others = rec_vs_others.loc[rec_vs_others['Dropbacks_y'] > 5]

rec_vs_others['Other Total EPA'] = rec_vs_others['Dropbacks_y'] * rec_vs_others['EPA per Dropback_y']

rec_vs_others = rec_vs_others.groupby(['Season', 'Team_x', 'Receiver_x', 'Passer_y', 'Dropbacks_x', ], as_index=False).agg({'EPA per Dropback_x':'mean', 'Other Total EPA':'sum','Dropbacks_y': 'sum'}) 

rec_vs_others['Other EPA per Dropback'] = rec_vs_others['Other Total EPA']/rec_vs_others['Dropbacks_y']

rec_vs_others['Receiver EPA/Other Receiver EPA Differential'] = rec_vs_others['EPA per Dropback_x'] - rec_vs_others['Other EPA per Dropback']

rec_vs_others = rec_vs_others.round(2)

rec_vs_others.columns = ['Season', 'Team', 'Receiver', 'QB', 'Dropbacks', 'Receiver/QB EPA', 'Rest of Team/QB Total EPA', 'Rest of Team Dropbacks', 'Rest of Team/QB EPA per dropback', 'Differential']

rec_vs_others = rec_vs_others[['Season', 'Team', 'Receiver', 'QB', 'Receiver/QB EPA', 'Dropbacks', 'Rest of Team/QB EPA per dropback', 'Rest of Team Dropbacks', 'Differential']]
rec_vs_others.sort_values('Season', ascending=False, inplace=True)
rec_vs_others.sort_values('QB', ascending=False, inplace=True)
rec_vs_others.sort_values('Receiver/QB EPA', ascending=False, inplace=True)

rec_vs_others
#rec_vs_others.sort_values('Differential', ascending=False, inplace=True)

,Season,Team,Receiver,QB,Receiver/QB EPA,Dropbacks,Rest of Team/QB EPA per dropback,Rest of Team Dropbacks,Differential
29,2021,PHI,D.Goedert,J.Hurts,0.58,64,0.13,323,0.45
14,2018,PHI,N.Agholor,N.Foles,0.51,36,0.12,157,0.39
7,2017,PHI,N.Agholor,C.Wentz,0.46,71,0.32,370,0.14
27,2020,PHI,T.Fulgham,C.Wentz,0.44,57,-0.02,365,0.46
30,2021,PHI,D.Smith,J.Hurts,0.38,91,0.15,296,0.23
23,2020,PHI,D.Goedert,C.Wentz,0.36,46,0.01,376,0.35
11,2018,PHI,D.Goedert,C.Wentz,0.36,33,0.27,364,0.09
9,2017,PHI,Z.Ertz,C.Wentz,0.36,85,0.34,356,0.02
6,2017,PHI,A.Jeffery,C.Wentz,0.33,107,0.35,334,-0.02
10,2018,PHI,A.Jeffery,C.Wentz,0.33,74,0.27,323,0.06


In [28]:
# Kirk Cousins 1 PM vs Later Times
reg_data["start_time"] = reg_data["start_time"].astype("|S").str.decode('utf-8') 

reg_data.loc[((reg_data.start_time.str.slice(stop=2) != '20'), 'start_time')] = 'Day Game'

reg_data.loc[(reg_data['start_time'] != 'Day Game', 'start_time')] = 'Night Game'

qb_time_data = reg_data.groupby(['passer', 'start_time'], as_index=False).agg({'qb_epa':'mean', 'cpoe':'mean',
                                                                                 'play_id':'count'})
kirk_time_data = qb_time_data.loc[qb_time_data.passer == 'K.Cousins']

kirk_time_data = kirk_time_data.round(2)

kirk_time_data.columns = ['Player', 'Kickoff Time', 'EPA per Dropback','CPOE','Dropbacks']

kirk_time_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

kirk_time_data

,Player,Kickoff Time,EPA per Dropback,CPOE,Dropbacks
330,K.Cousins,Day Game,0.14,2.87,3716
331,K.Cousins,Night Game,0.09,1.79,1070


In [29]:
qb_time_data = qb_time_data.round(2)

qb_time_data.columns = ['Player', 'Kickoff Time', 'EPA per Dropback','CPOE','Dropbacks']

qb_time_data = qb_time_data.loc[qb_time_data['Dropbacks'] > 250]

qb_time_data.sort_values('EPA per Dropback', ascending=False, inplace=True)

early_time_data = qb_time_data[qb_time_data['Kickoff Time'] == 'Day Game']

early_time_data.head(25)

,Player,Kickoff Time,EPA per Dropback,CPOE,Dropbacks
444,P.Mahomes,Day Game,0.31,3.61,1790
518,T.Brady,Day Game,0.25,1.81,5253
22,A.Rodgers,Day Game,0.24,3.23,4473
446,P.Manning,Day Game,0.23,4.35,1601
257,J.Garoppolo,Day Game,0.23,2.35,1175
142,D.Brees,Day Game,0.21,3.79,4798
188,D.Watson,Day Game,0.19,4.12,1937
555,T.Romo,Day Game,0.18,2.91,1786
452,P.Rivers,Day Game,0.18,3.33,5417
358,L.Jackson,Day Game,0.17,0.42,1206


In [30]:
later_time_data = qb_time_data[qb_time_data['Kickoff Time'] == 'Night Game']

later_time_data.head(25)

,Player,Kickoff Time,EPA per Dropback,CPOE,Dropbacks
222,J.Allen,Night Game,0.30,6.68,330
445,P.Mahomes,Night Game,0.27,2.10,895
189,D.Watson,Night Game,0.27,4.67,293
23,A.Rodgers,Night Game,0.25,4.17,2106
447,P.Manning,Night Game,0.25,4.70,782
143,D.Brees,Night Game,0.23,5.41,1503
359,L.Jackson,Night Game,0.21,2.06,458
409,M.Ryan,Night Game,0.20,5.53,1278
398,M.Mariota,Night Game,0.19,6.09,334
64,B.Roethlisberger,Night Game,0.18,2.89,1638


In [2]:
qb_time_splits = early_time_data.merge(later_time_data, on='Player')

qb_time_splits['Late vs. Early Differential'] = qb_time_splits['EPA per Dropback_x'] - qb_time_splits['EPA per Dropback_y']

qb_time_splits.columns = ['Player', 'Early Times', 'Early EPA', 'Early CPOE', 'Early Dropbacks', 'Late Times', 'Late EPA', 'Late CPOE', 'Late Dropbacks', 'Late vs. Early Differential']

qb_time_splits = qb_time_splits[['Player', 'Early EPA', 'Early CPOE', 'Early Dropbacks', 'Late EPA', 'Late CPOE', 'Late Dropbacks', 'Late vs. Early Differential']]

qb_time_splits.sort_values('Late vs. Early Differential', ascending=False, inplace=True)

qb_time_splits.head(25)

NameError: name 'early_time_data' is not defined

In [32]:
qb_time_splits.tail(10)

,Player,Early EPA,Early CPOE,Early Dropbacks,Late EPA,Late CPOE,Late Dropbacks,Late vs. Early Differential
45,B.Osweiler,-0.03,-2.97,979,0.02,-2.97,372,-0.05
33,J.Goff,0.06,-1.56,2552,0.11,1.50,742,-0.05
13,M.Ryan,0.15,2.27,6174,0.20,5.53,1278,-0.05
20,C.Palmer,0.10,1.44,3004,0.16,3.90,509,-0.06
35,J.Flacco,0.03,-1.23,4260,0.09,0.56,1051,-0.06
40,C.Keenum,0.01,-2.20,2055,0.07,1.42,423,-0.06
6,D.Watson,0.19,4.12,1937,0.27,4.67,293,-0.08
43,J.Cutler,0.00,-0.12,2309,0.09,2.64,843,-0.09
32,M.Mariota,0.07,0.09,1859,0.19,6.09,334,-0.12
15,J.Allen,0.13,0.70,2017,0.30,6.68,330,-0.17


In [61]:
clutch_situations = data[['play_id', 'season', 'passer', 'posteam', 'defteam', 'qtr', 'game_seconds_remaining', 'score_differential', 'qb_epa', 'cpoe']]

clutch_situations = clutch_situations.loc[(((clutch_situations['score_differential'] < 15) & (clutch_situations['score_differential'] > -14)))]

clutch_situations = clutch_situations.loc[(clutch_situations['game_seconds_remaining'] < 601) | (clutch_situations['qtr'] == 5)]

game_out_of_hand = clutch_situations.loc[((clutch_situations['score_differential'] > 8) | (clutch_situations['score_differential'] < -8))]

game_out_of_hand = game_out_of_hand.loc[game_out_of_hand['game_seconds_remaining'] < 91]

# Remove garbage time
valid_clutch_plays = pd.concat([clutch_situations,game_out_of_hand]).drop_duplicates(keep=False)

valid_clutch_plays = valid_clutch_plays.groupby(['passer'], as_index=False).agg({'qb_epa':'mean', 'cpoe':'mean', 'play_id':'count'})

valid_clutch_plays = valid_clutch_plays.round(2)
valid_clutch_plays = valid_clutch_plays[valid_clutch_plays.play_id > 75]

valid_clutch_plays.columns = ['QB', 'EPA', 'CPOE', 'Dropbacks']

valid_clutch_plays.sort_values('EPA', ascending=False, inplace=True)

valid_clutch_plays.head(30)

,QB,EPA,CPOE,Dropbacks
104,J.Locker,0.39,-0.39,84
72,D.Watson,0.38,7.24,292
159,P.Mahomes,0.34,4.25,317
136,M.Hasselbeck,0.33,4.17,128
9,A.Rodgers,0.27,2.44,772
200,T.Siemian,0.26,2.65,142
198,T.Romo,0.26,4.40,398
95,J.Freeman,0.26,-1.80,153
83,J.Burrow,0.24,6.19,148
172,R.Wilson,0.22,5.31,903


In [59]:
valid_clutch_plays.tail(30)

,QB,EPA,CPOE,Dropbacks
132,M.Barkley,0.03,-6.64,77
167,R.Grossman,0.02,-0.59,99
1,A.Dalton,0.02,-3.30,779
125,K.Kolb,0.01,-6.42,86
119,K.Allen,0.01,9.39,78
152,M.Vick,0.01,-0.27,147
101,J.Hurts,0.00,-5.26,81
109,J.McCown,-0.01,2.14,305
147,M.Sanchez,-0.01,-2.35,188
40,C.Kaepernick,-0.01,-4.35,221


In [60]:
eagles_clutch_plays = valid_clutch_plays[(valid_clutch_plays['QB'] == 'C.Wentz') | (valid_clutch_plays['QB'] == 'N.Foles') | (valid_clutch_plays['QB'] == 'J.Hurts')]

eagles_clutch_plays

,QB,EPA,CPOE,Dropbacks
154,N.Foles,0.07,0.09,286
52,C.Wentz,0.05,-2.43,499
101,J.Hurts,0.00,-5.26,81


In [63]:
ot_data = data[['play_id', 'season', 'passer', 'posteam', 'defteam', 'qtr', 'game_seconds_remaining', 'score_differential', 'qb_epa', 'cpoe']]

ot_data = ot_data.loc[(ot_data['qtr'] == 5)]

ot_data = ot_data.groupby(['passer'], as_index=False).agg({'qb_epa':'mean', 'cpoe':'mean', 'play_id':'count'})

ot_data = ot_data.round(2)
ot_data = ot_data[valid_clutch_plays.play_id > 75]

ot_data.columns = ['QB', 'EPA', 'CPOE', 'Dropbacks']

ot_data.sort_values('EPA', ascending=False, inplace=True)

ot_data.head(30)

AttributeError: 'DataFrame' object has no attribute 'play_id'